In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
DATA_PATH = "../data/arxiv_data.csv"
os.path.isfile(DATA_PATH)

True

In [4]:
df = pd.read_csv(DATA_PATH)
df.head()

,terms,titles,abstracts
0,['cs.LG'],Multi-Level Attention Pooling for Graph Neural...,Graph neural networks (GNNs) have been widely ...
1,"['cs.LG', 'cs.AI']",Decision Forests vs. Deep Networks: Conceptual...,Deep networks and decision forests (such as ra...
2,"['cs.LG', 'cs.CR', 'stat.ML']",Power up! Robust Graph Convolutional Network v...,Graph convolutional networks (GCNs) are powerf...
3,"['cs.LG', 'cs.CR']",Releasing Graph Neural Networks with Different...,With the increasing popularity of Graph Neural...
4,['cs.LG'],Recurrence-Aware Long-Term Cognitive Network f...,Machine learning solutions for pattern classif...


In [5]:
len(df)

56181

In [7]:
df.isna().sum()

terms        0
titles       0
abstracts    0
dtype: int64

In [9]:
title_group = df.groupby("titles")

In [15]:
len(title_group)

41105

There are same titles repeated multiple times. We need to remove them.

In [18]:
abstract_group = df.groupby("abstracts")
len(abstract_group)

41115

In [22]:
df_deduped = df.drop_duplicates()
df_deduped.head()

,terms,titles,abstracts
0,['cs.LG'],Multi-Level Attention Pooling for Graph Neural...,Graph neural networks (GNNs) have been widely ...
1,"['cs.LG', 'cs.AI']",Decision Forests vs. Deep Networks: Conceptual...,Deep networks and decision forests (such as ra...
2,"['cs.LG', 'cs.CR', 'stat.ML']",Power up! Robust Graph Convolutional Network v...,Graph convolutional networks (GCNs) are powerf...
3,"['cs.LG', 'cs.CR']",Releasing Graph Neural Networks with Different...,With the increasing popularity of Graph Neural...
4,['cs.LG'],Recurrence-Aware Long-Term Cognitive Network f...,Machine learning solutions for pattern classif...


In [23]:
len(df_deduped)

41127

In [25]:
len(df_deduped.groupby("titles"))

41105

In [29]:
# we don't need terms so dropping it
df_deduped = df_deduped[["titles", "abstracts"]]

In [30]:
df_deduped.to_csv("../data/arxiv_data_new.csv", index=False)

## Loading as HF Dataset

In [31]:
from datasets import load_dataset

In [61]:
dataset = load_dataset("csv", data_files="../data/arxiv_data_new.csv")
dataset = dataset["train"]

Using custom data configuration default-afb449b07e2294db
Found cached dataset csv (/home/ayushthakur/.cache/huggingface/datasets/csv/default-afb449b07e2294db/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [62]:
dataset

Dataset({
    features: ['titles', 'abstracts'],
    num_rows: 41127
})

In [63]:
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['titles', 'abstracts'],
        num_rows: 32901
    })
    test: Dataset({
        features: ['titles', 'abstracts'],
        num_rows: 8226
    })
})

In [64]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [65]:
PREFIX = "summarize: "


# find title and abstract token lenghts from EDA
def preprocess_function(examples):
    inputs = [PREFIX + doc for doc in examples["abstracts"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["titles"], max_length=24, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [66]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/33 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [67]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['titles', 'abstracts', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 32901
    })
    test: Dataset({
        features: ['titles', 'abstracts', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8226
    })
})

In [68]:
tokenized_dataset = tokenized_dataset.remove_columns(['titles', 'abstracts'])
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 32901
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8226
    })
})

In [ ]:
tokenized_dataset.to_tf_


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [59]:
import tensorflow as tf

In [60]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(tokenized_dataset["train"])
))

ValueError: dictionary update sequence element #0 has length 5; 2 is required

In [ ]:
def preprocess_function(examples):
    inputs = [PREFIX + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=48, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = billsum.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

tf_train_set = model.prepare_tf_dataset(
    tokenized_billsum["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_billsum["test"],
    shuffle=False,
    batch_size=8,
    collate_fn=data_collator,
)